<a href="https://colab.research.google.com/github/josepmatias/ONE/blob/main/auto_campanhas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
pip install pyxlsb

In [5]:
#nomear os ficheiros

#activacoes="auto_listagemactivacoes.xlsb"
#auto="03.2024 - Auto Upsell_Prospect_Rec_Cancelados_formulas.xlsb"
modelo_upfront="modelo upfront.xlsb"
activacoes="ONE_Ativações_Serviços_Marcações_Migrações_mar24.xlsb"

In [6]:
#carregar ficheiro de activações e auto

from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()

Saving ONE_Ativações_Serviços_Marcações_Migrações_mar24.xlsb to ONE_Ativações_Serviços_Marcações_Migrações_mar24.xlsb


Saving modelo upfront.xlsb to modelo upfront.xlsb


In [ ]:
#from google.colab import files
#uploaded3 = files.upload()

Saving modelo upfront.xlsb to modelo upfront.xlsb


In [7]:
#ler ficheiro db_upfront
#db_upfront usada para o valor upfront €, excepto quando análise daf é "DTH para Cabo" e "Pré-Pago"
db_upfront=pd.read_excel(modelo_upfront, sheet_name="modelo upfront", header=1).iloc[:, [2, 3, 4]]

db_upfront.head()


,Sub-Área de Negócio,Upfront,Pts GCO
0,ALARME,25.0,depende da campanha
1,CABO,10.0,depende da campanha
2,Desporto,2.0,0
3,DTH,10.0,depende da campanha
4,DTH para Cabo,5.0,1.25


In [8]:
#ler ficheiro db_pontos_gco
#db_pontos_gco é usada para fixar pontos (coluna pts GCO) excepto quando a análise daf é "Comissionada Navegação Segura", "Comissionado Seguro" \
#"Comissionado Router 6.0", ""Comissionado Power wi-fi""
db_pontos_gco=pd.read_excel(modelo_upfront, sheet_name="modelo upfront", header=1).iloc[:, 8:10]
db_pontos_gco.head()

,Campanha,pts GCO
0,lm_churn,0.75
1,lm_churn_ac,0.75
2,prospect_alarmes,3.00
3,prospect_blitz_blm,2.00
4,prospect_blitz_wow,2.00


In [9]:
#ler ficheiro da listagem de activações
auto_db=pd.read_excel(activacoes, sheet_name="Detalhe")
print(auto_db.shape)
auto_db.head()

(8779, 47)


,Mês de Fecho,Order Item,Chave,SFID,Comissionamento,Sub-Área de Negócio,Service Account,Customer Account (SA),CA&UA,Alojamento Id,...,Notas DS,Upfront €,pts GCO,GCO €,Total €,Final,Analise DAF,Pagar,Conclusão SDD,Análise SDD
0,45352,1-KLM2AVR,40168940583780086295,L.G01.UP1.043,ONE,CABO,S965395918,C145652802,C1456528027901704365,8996880.0,...,Uso da campanha final ou wise dará pts de GCO ...,10.0,3.00,2.0,12.0,ok,Comissionado,Sim,NaN,NaN
1,45352,1-KNXHCJ1,40168940583780047659,L.G01.UP1.011,ONE,CABO,S825253392,C172136602,C1721366021255016819,2122847.0,...,NaN,10.0,3.00,2.0,12.0,ok,Comissionado,Sim,NaN,NaN
2,45352,1-KMQRCQB,20544240583580122712,B.C01.UP1.287,ONE,CABO,S846897051,C173519001,C1735190017914773340,2244259.0,...,NaN,10.0,3.00,2.0,12.0,ok,Comissionado,Sim,NaN,NaN
3,45352,1-KLG8LET,40168940583770080648,L.G01.UP1.627,ONE,CABO,S965388611,C174106801,C17410680110012563678,4146708.0,...,NaN,10.0,2.00,-7.5,2.5,ok,Comissionado Blitz BLM,Sim,NaN,NaN
4,45352,1-KOOQNHS,40168941003680079762,L.G01.RMC.006,ONE,CABO,S848815192,C225385001,C22538500110016569851,1565886.0,...,NaN,5.0,1.25,1.0,6.0,ok,Comissionado DTH para Cabo,Sim,NaN,NaN


In [59]:
#encontrar valores upfront para operador - coluna "Sub-Área de Negócio"

def upfront1 (df, df2, col="Pagar", col2="Analise DAF"):
  #quando análise daf não é "DTH para Cabo" nem "Pré-Pago"
  df1=df.loc[(df[col]=="Sim") & (~df[col2].isin(["DTH para Cabo", "Pré-Pago"])), ["Sub-Área de Negócio"]]
  df1["upfront"]=df1.merge(df2, on="Sub-Área de Negócio", how="right").iloc[:, 1]

  #quando análise daf é dth para cabo
  df3=pd.DataFrame(df.loc[(df[col]=="Sim") & (df[col2].str.find("DTH para Cabo")!=-1), ["Sub-Área de Negócio", "Analise DAF"]].\
  apply(lambda i: i[0]+i[1], axis=1))

  df3.merge(df2, on="Sub-Área de Negócio", how="right").iloc[:, 1]

  return df2

upfront1(auto_db, db_upfront.iloc[:, [0, 1]])



#auto_db.loc[(auto_db["Pagar"]=="Sim") & (~auto_db["Analise DAF"].isin(["DTH para Cabo", "Pré-Pago"])), "Sub-Área de Negócio"]


#auto_db[["Sub-Área de Negócio"]].merge(db_upfront.iloc[:, [0, 1]], on="Sub-Área de Negócio")

#db_upfront.iloc[:, [0, 1]]

KeyError: 'Sub-Área de Negócio'

In [52]:
#auto_db.loc[(auto_db["Pagar"]=="Sim") & (auto_db["Analise DAF"].str.startswith("DTH para Cabo")==True), ["Sub-Área de Negócio", "Analise DAF"]]

auto_db.loc[auto_db["Analise DAF"].str.find("DTH para Cabo")!=-1,["Sub-Área de Negócio", "Analise DAF"]].apply(lambda i: i[0] + i[1], axis=1)



4       CABOComissionado DTH para Cabo
7       CABOComissionado DTH para Cabo
10      CABOComissionado DTH para Cabo
14      CABOComissionado DTH para Cabo
24      CABOComissionado DTH para Cabo
                     ...              
8774    CABOComissionado DTH para Cabo
8775     NETComissionado DTH para Cabo
8776       PÓS-PAGOMóvel DTH para Cabo
8777    VOIPComissionado DTH para Cabo
8778    VOIPComissionado DTH para Cabo
Length: 124, dtype: object